In [72]:
import csv
import re

In [123]:
# counts number of rows
with open('yourFile.csv', "r", encoding="utf-8") as file:
    readCSV = csv.reader(file)
    lines = len(list(readCSV))
    
file.close()

In [124]:
def saveSameBankName(equalBanks, rowBefore, row):
    if not equalBanks:
            equalBanks += [rowBefore]
    equalBanks += [row]
    return equalBanks

def saveSameBICs(equalBanks, presentBIC):
    # check for emtpy BIC (has to be at least one BIC!)
    for line in equalBanks: 
        if line[2] != "":             
            presentBIC += [line]
    
    return presentBIC

def leaveOutRedundantLine(equalBanks, presentBIC):
    for line in equalBanks:
        countingPrintinginFile = 0
        differentBankCode = 0

        # does not print redundant lines
        for bicLine in presentBIC:
            if countingPrintinginFile == 1:
                print()
            elif line[5] == bicLine[5] and line[2] != "":   # bankcode same, BIC not empty
                countingPrintinginFile = 1                  # also: different bankcode, BIC not empty
                writer.writerow(line)
            elif line[5] == bicLine[5] and line[2] == "":   # bankcode same, BIC empty
                countingPrintinginFile = 1
            elif line[5] != bicLine[5] and line[2] == "":   # bankcode different, BIC empty
                differentBankCode = differentBankCode + 1
            
            if differentBankCode == len(presentBIC):
                writer.writerow(line)

    
with open('yourFile.csv', "r", encoding="utf-8") as file:
    readCSV = csv.reader(file)
    
    # output will be saved in new file
    noRedundancy = open('resultingFile_noRedundancy.csv', 'w')
    writer = csv.writer(noRedundancy)
    bankName = ""; bic = ""; bankCode = ""; rowBefore = "" 
    equalBanks = []
    counter = 0
    presentBIC = []
    
    for row in readCSV:
        counter = counter + 1
        
        if not equalBanks:
            if rowBefore:
                equalBanks = [rowBefore]
        
        # same bank name
        if bankName == row[1]:
            equalBanks = saveSameBankName(equalBanks, rowBefore, row)
            
        elif bankName in row[1] and ("(Gf P2)" in row[1] or "(Gf P3)" in row[1]):
            equalBanks = saveSameBankName(equalBanks, rowBefore, row)
        
        else:
            if "(Gf P2)" in bankName:
                bankName = re.sub("Gf P2", "", bankName)
                bankName = re.sub("[()]", "", bankName)
                
            elif "(Gf P3)" in bankName:
                bankName = re.sub("Gf P3", "", bankName)
                bankName = re.sub("[()]", "", bankName)
            
            # looks in array of same banks for redundancy
            if re.search(bankName, str(equalBanks)):
                presentBIC = saveSameBICs(equalBanks, presentBIC)
                leaveOutRedundantLine(equalBanks, presentBIC)

            # different bank names
            else: 
                writer.writerow(rowBefore)

            equalBanks = []    
            presentBIC = []
               
        bankName = row[1]
        bic = row[2]
        bankCode = row[5]
        rowBefore = row
        
        
         # handle last row
        if counter == lines:
            if re.search(bankName, str(equalBanks)):    
                presentBIC = saveSameBICs(equalBanks, presentBIC)
                leaveOutRedundantLine(equalBanks, presentBIC)

            # different bank names
            else: 
                writer.writerow(rowBefore)

            equalBanks = [] 
            presentBIC = []